# Deep Reinforcement Learning using AlphaZero methodology

Please see https://applied-data.science/blog/how-to-build-your-own-alphazero-ai-using-python-and-keras/ for further notes on the codebase

## 1. First load the core libraries

In [5]:
# -*- coding: utf-8 -*-
# %matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
np.set_printoptions(suppress=True)

from shutil import copyfile
import random
from importlib import reload


from keras.utils import plot_model

from game import Game, GameState
from agent import Agent
from memory import Memory
from model import Residual_CNN
from funcs import playMatches, playMatchesBetweenVersions

import loggers as lg

from settings import run_folder, run_archive_folder
import initialise
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Now run this block to start the learning process

This block loops for ever, continually learning from new game data.

The current best model and memories are saved in the run folder so you can kill the process and restart from the last checkpoint.

## The following panels are not involved in the learning process

### Play matches between versions (use -1 for human player)

In [9]:
lg.logger_main.info('=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*')
lg.logger_main.info('=*=*=*=*=*=.      NEW LOG      =*=*=*=*=*')
lg.logger_main.info('=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*=*')

env = Game()

# If loading an existing neural network, copy the config file to root
if initialise.INITIAL_RUN_NUMBER != None:
    copyfile(run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + '/config.py', './config.py')

import config

######## LOAD MEMORIES IF NECESSARY ########

if initialise.INITIAL_MEMORY_VERSION == None:
    memory = Memory(config.MEMORY_SIZE)
else:
    print('LOADING MEMORY VERSION ' + str(initialise.INITIAL_MEMORY_VERSION) + '...')
    memory = pickle.load( open( run_archive_folder + env.name + '/run' + str(initialise.INITIAL_RUN_NUMBER).zfill(4) + "/memory/memory" + str(initialise.INITIAL_MEMORY_VERSION).zfill(4) + ".p",   "rb" ) )

######## LOAD MODEL IF NECESSARY ########

# create an untrained neural network objects from the config file
current_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, env.input_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)
best_NN = Residual_CNN(config.REG_CONST, config.LEARNING_RATE, env.input_shape,   env.action_size, config.HIDDEN_CNN_LAYERS)

#If loading an existing neural netwrok, set the weights from that model
if initialise.INITIAL_MODEL_VERSION != None:
    best_player_version  = initialise.INITIAL_MODEL_VERSION
    print('LOADING MODEL VERSION ' + str(initialise.INITIAL_MODEL_VERSION) + '...')
    m_tmp = best_NN.read(env.name, initialise.INITIAL_RUN_NUMBER, best_player_version)
    current_NN.model.set_weights(m_tmp.get_weights())
    best_NN.model.set_weights(m_tmp.get_weights())
#otherwise just ensure the weights on the two players are the same
else:
    best_player_version = 0
    best_NN.model.set_weights(current_NN.model.get_weights())

#copy the config file to the run folder
copyfile('./config.py', run_folder + 'config.py')
# plot_model(current_NN.model, to_file=run_folder + 'models/model.png', show_shapes = True)

print('\n')

######## CREATE THE PLAYERS ########

current_player = Agent('current_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, current_NN)
best_player = Agent('best_player', env.state_size, env.action_size, config.MCTS_SIMS, config.CPUCT, best_NN)
#user_player = User('player1', env.state_size, env.action_size)
iteration = 0

while 1:

    iteration += 1
    reload(lg)
    reload(config)
    
    print('ITERATION NUMBER ' + str(iteration))
    
    lg.logger_main.info('BEST PLAYER VERSION: %d', best_player_version)
    print('BEST PLAYER VERSION ' + str(best_player_version))

    ######## SELF PLAY ########
    print('SELF PLAYING ' + str(config.EPISODES) + ' EPISODES...')
    _, memory, _, _ = playMatches(best_player, best_player, config.EPISODES, lg.logger_main, turns_until_tau0 = config.TURNS_UNTIL_TAU0, memory = memory)
    print('\n')
    
    memory.clear_stmemory()
    
    if len(memory.ltmemory) >= config.MEMORY_SIZE:

        ######## RETRAINING ########
        print('RETRAINING...')
        current_player.replay(memory.ltmemory)
        print('')

        if iteration % 5 == 0:
            pickle.dump( memory, open( run_folder + "memory/memory" + str(iteration).zfill(4) + ".p", "wb" ) )

        lg.logger_memory.info('====================')
        lg.logger_memory.info('NEW MEMORIES')
        lg.logger_memory.info('====================')
        
        memory_samp = random.sample(memory.ltmemory, min(1000, len(memory.ltmemory)))
        
        for s in memory_samp:
            current_value, current_probs, _ = current_player.get_preds(s['state'])
            best_value, best_probs, _ = best_player.get_preds(s['state'])

            lg.logger_memory.info('MCTS VALUE FOR %s: %f', s['playerTurn'], s['value'])
            lg.logger_memory.info('CUR PRED VALUE FOR %s: %f', s['playerTurn'], current_value)
            lg.logger_memory.info('BES PRED VALUE FOR %s: %f', s['playerTurn'], best_value)
            lg.logger_memory.info('THE MCTS ACTION VALUES: %s', ['%.2f' % elem for elem in s['AV']]  )
            lg.logger_memory.info('CUR PRED ACTION VALUES: %s', ['%.2f' % elem for elem in  current_probs])
            lg.logger_memory.info('BES PRED ACTION VALUES: %s', ['%.2f' % elem for elem in  best_probs])
            lg.logger_memory.info('ID: %s', s['state'].id)
            lg.logger_memory.info('INPUT TO MODEL: %s', current_player.model.convertToModelInput(s['state']))

            s['state'].render(lg.logger_memory)
            
        ######## TOURNAMENT ########
        print('TOURNAMENT...')
        scores, _, points, sp_scores = playMatches(best_player, current_player, config.EVAL_EPISODES, lg.logger_tourney, turns_until_tau0 = 0, memory = None)
        print('\nSCORES')
        print(scores)
        print('\nSTARTING PLAYER / NON-STARTING PLAYER SCORES')
        print(sp_scores)
        #print(points)

        print('\n\n')

        
        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            best_player_version = best_player_version + 1
            best_NN.model.set_weights(current_NN.model.get_weights())
            best_NN.write(env.name, best_player_version)

    else:
        print('MEMORY SIZE: ' + str(len(memory.ltmemory)))



ITERATION NUMBER 1
BEST PLAYER VERSION 0
SELF PLAYING 5 EPISODES...
1 best_player white
White's turn
g1h3, g1f3, b1c3, b1a3, h2h3, g2g3, f2f3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4, g2g4, f2f4, e2e4, d2d4, c2c4, b2b4, a2a4, 
495, 494, 130, 129, 1095, 1022, 949, 876, 803, 730, 657, 584, 1096, 1023, 950, 877, 804, 731, 658, 585, 
White's Move  :1
0
1022
g2g3
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . P .
P P P P P P . P
R N B Q K B N R

Black's turn
g8h6, g8f6, b8c6, b8a6, h7h6, g7g6, f7f6, e7e6, d7d6, c7c6, b7b6, a7a6, h7h5, g7g5, f7f5, e7e5, d7d5, c7c5, b7b5, a7a5, 
4585, 4584, 4220, 4219, 4022, 3949, 3876, 3803, 3730, 3657, 3584, 3511, 4023, 3950, 3877, 3804, 3731, 3658, 3585, 3512, 
Black's Move: 2
0
3511
a7a6
r n b q k b n r
. p p p p p p p
p . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . P .
P P P P P P . P
R N B Q K B N R

White's turn
g1h3, g1f3, f1h3, f1g2, b1c3, b1a3, g3g4, h2h3, f2f3, e2e3, d2d3, c2c3, b2b3, a2a3, h2h4

Black's turn
h8g8, d8c7, d8b6, d8a5, b8a6, a8a7, a8a6, a8a5, h6g8, h6f5, h6g4, g7g6, f7f6, e7e6, d7d6, b7b6, c6c5, g7g5, f7f5, e7e5, d7d5, b7b5, 
4620, 4349, 4350, 4351, 4219, 4095, 4096, 4097, 3487, 3492, 3489, 3949, 3876, 3803, 3730, 3584, 3073, 3950, 3877, 3804, 3731, 3585, 
Black's Move: 20
0
3804
e7e5
r n b q k b . r
. p . p . p p p
. . p . . . . n
. . . . p . . .
p . . . . P . .
P . N . . . P P
. P P Q P . R .
R . B . K B . .

White's turn
c3d5, c3b5, c3e4, c3a4, c3a2, c3d1, c3b1, g2h2, g2f2, g2g1, d2d7, d2d6, d2d5, d2d4, d2e3, d2d3, d2d1, e1f2, e1d1, a1a2, a1b1, f4e5, f4f5, h3h4, g3g4, e2e3, b2b3, e2e4, b2b4, 
1371, 1370, 1376, 1333, 1375, 1373, 1372, 1036, 1043, 1029, 807, 806, 805, 804, 838, 803, 810, 327, 313, 0, 14, 2137, 2117, 1679, 1606, 876, 657, 877, 658, 
White's Move  :21
0
807
d2d7
r n b q k b . r
. p . Q . p p p
. . p . . . . n
. . . . p . . .
p . . . . P . .
P . N . . . P P
. P P . P . R .
R . B . K B . .

Black's turn
e8d7, d8d7, c8d7, b8d7, 
4422, 4314, 4261, 4187

Black's turn
g8h8, f8e7, f8d6, e8f7, e8e7, e8d7, d8c8, d8e7, d8d7, d8c7, d8d6, d8b6, d8d5, d8a5, d8d4, d8d3, d8d2, d8d1, b8d7, b8c6, b8a6, a8a7, a8a6, a8a5, h6f7, h6g4, f5c8, f5d7, f5g6, f5e6, f5g4, f5e4, f5h3, f5d3, f5c2, e5f4, g7g6, b7b6, e5e4, c5c4, g7g5, b7b5, 
4540, 4495, 4496, 4407, 4387, 4422, 4328, 4334, 4314, 4349, 4315, 4350, 4316, 4351, 4317, 4318, 4319, 4320, 4187, 4220, 4219, 4095, 4096, 4097, 3450, 3489, 2731, 2730, 2736, 2721, 2728, 2743, 2751, 2744, 2745, 2655, 3949, 3584, 2635, 2489, 3950, 3585, 
Black's Move: 36
2
4540
g8h8
r n . q k b . r
. p . . . . p p
. . . . . p . n
. . p . p b . .
p . . . . P . .
P . N . P . P P
. P P . . R B .
. R B . . K . .

White's turn
c3d5, c3b5, c3e4, c3a4, c3e2, c3a2, c3d1, g2b7, g2c6, g2d5, g2e4, g2f3, g2h1, f2f3, f2e2, f2d2, f1e2, f1g1, f1e1, c1d2, b1a1, f4e5, h3h4, g3g4, e3e4, b2b3, b2b4, 
1371, 1370, 1376, 1333, 1340, 1375, 1373, 1054, 1053, 1052, 1051, 1042, 1049, 949, 970, 971, 385, 379, 386, 181, 94, 2137, 1679, 1606, 1460, 657, 6

White's turn
d7e8, d7c8, d7e6, d7c6, d7f5, d7b5, d7a4, d6f6, d6e6, d6c6, d6b6, d6a6, d6d5, d6d4, d6d3, d6d2, d6d1, f2g2, f2e2, f2g1, f2f1, f2e1, a2b4, a2c3, c1d2, b1a1, h3h4, g3g4, c2c3, b2b3, c2c4, b2b4, 
3758, 3743, 3750, 3765, 3773, 3766, 3767, 3154, 3153, 3160, 3161, 3162, 3146, 3147, 3148, 3149, 3150, 963, 970, 976, 956, 991, 641, 646, 181, 94, 1679, 1606, 730, 657, 731, 658, 
White's Move  :51
3
3765
d7c6
. n . . . b . r
. p . . k q p p
. . B R . p . n
r . p . . b . .
p . . . p P . .
P . . . P . P P
N P P . . K . .
. R B . . . . .

Black's turn
h8g8, b8d7, b8c6, b8a6, f7g8, f7e8, f7g6, f7e6, f7h5, f7d5, f7c4, f7b3, f7a2, e7d6, h6g8, h6g4, f5c8, f5d7, f5g6, f5e6, f5g4, f5h3, a5a8, a5a7, a5a6, a5b5, b7c6, g7g6, b7b6, c5c4, g7g5, b7b5, 
4620, 4187, 4220, 4219, 3904, 3889, 3896, 3911, 3919, 3912, 3913, 3914, 3915, 3838, 3487, 3489, 2731, 2730, 2736, 2721, 2728, 2751, 2338, 2337, 2336, 2350, 3604, 3949, 3584, 2489, 3950, 3585, 
Black's Move: 52
4
3912
f7d5
. n . . . b . r
. p . . k . 

Black's turn
h8g8, h8h7, h8h6, e7e8, e7d8, e7f7, e7d7, e7e6, e7d6, c6d8, c6b8, c6a7, c6e5, c6a5, c6d4, c6b4, f5c8, f5h7, f5d7, f5g6, f5e6, f5g4, f5h3, c5e5, c5d5, c5b5, c5a5, h2g4, h2f3, h2f1, c4b3, a4b3, g7g6, b6b5, h5h4, c4c3, g7g5, 
4620, 4606, 4607, 3796, 3816, 3810, 3817, 3803, 3838, 3123, 3122, 3085, 3092, 3127, 3125, 3124, 2731, 2737, 2730, 2736, 2721, 2728, 2751, 2497, 2496, 2503, 2504, 1151, 1114, 1156, 1940, 1779, 3949, 3000, 2854, 1905, 3950, 
Black's Move: 68
2
3122
c6b8
. n . . . b . r
. . . . k . p .
. p . . . p . .
. . r . . b . p
p . p . p P . .
P R . . P . P P
N P P . . . . n
R . B . . . K .

White's turn
b3b6, b3b5, b3b4, b3d3, b3c3, a2b4, a2c3, g1h2, g1g2, g1f2, g1h1, c1d2, a1b1, h3h4, g3g4, c2c3, 
1243, 1242, 1241, 1256, 1255, 641, 646, 473, 438, 458, 452, 181, 14, 1679, 1606, 730, 
White's Move  :69
3
646
a2c3
. n . . . b . r
. . . . k . p .
. p . . . p . .
. . r . . b . p
p . p . p P . .
P R N . P . P P
. P P . . . . n
R . B . . . K .

Black's turn
h8g8, h8h7, h8h

White's turn
d5e7, d5c7, d5f6, d5b6, d5b4, c3c4, c3d3, c3b3, h1h2, h1g2, h1g1, c1d2, a1a2, a1b1, g5f6, g5g6, h3h4, b2b3, b2b4, 
2612, 2611, 2617, 2574, 2616, 1314, 1328, 1335, 511, 531, 532, 181, 0, 14, 2794, 2774, 1679, 657, 658, 
White's Move  :85
0
1679
h3h4
. n . . . . . .
. . . k b . p r
. . . . . p . .
. p . N . b P p
p . p . p P . P
P . R . P . . .
. P P . . . . n
R . B . . . . K

Black's turn
b8c6, b8a6, h7h8, h7h6, e7f8, e7d8, e7d6, e7c5, e7b4, e7a3, d7e8, d7d8, d7c8, d7e6, d7d6, d7c6, f5g6, f5e6, f5g4, f5h3, h2g4, h2f3, h2f1, f6g5, g7g6, b5b4, 
4220, 4219, 4015, 4022, 3831, 3816, 3838, 3839, 3840, 3841, 3758, 3723, 3743, 3750, 3730, 3765, 2736, 2721, 2728, 2751, 1151, 1114, 1156, 3312, 3949, 2416, 
Black's Move: 86
0
3841
e7a3
. n . . . . . .
. . . k . . p r
. . . . . p . .
. p . N . b P p
p . p . p P . P
b . R . P . . .
. P P . . . . n
R . B . . . . K

White's turn
d5e7, d5c7, d5f6, d5b6, d5b4, c3c4, c3d3, c3b3, c3a3, h1h2, h1g2, h1g1, c1d2, a1a3, a1a2, a1b1, g5f6, b2a3, g5g

IndexError: list index out of range

In [ ]:
from game import Game
from funcs import playMatchesBetweenVersions
import loggers as lg

env = Game()
playMatchesBetweenVersions(env, 1, 1, 1, 10, lg.logger_tourney, 0)

### Pass a particular game state through the neural network (setup below for Connect4)

In [ ]:
gs = GameState(np.array([
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,
    0,0,0,0,0,0,0
]), 1)

preds = current_player.get_preds(gs)

print(preds)

### See the layers of the current neural network

In [ ]:
from keras.utils import plot_model
plot_model(current_NN.model, to_file=run_folder + 'models/model.png', show_shapes = True)

In [ ]:
current_player.model.viewLayers()

### Output a diagram of the neural network architecture

In [ ]:
 print('MEMORY SIZE: ' + str(len(memory.ltmemory)))